In [1]:
import json
from pathlib import Path

import numpy as np
from tqdm.auto import tqdm

from src import dino

def MSE(img1, img2):
    assert img1.shape == img2.shape, f"{img1.shape} != {img2.shape}"
    return np.mean((img1 - img2) ** 2)

tonemapper_brightness_losses = {}
for image_name, (width, height) in [
    ("indoor", (600, 450)),
    ("outdoor", (600, 900)),
    ("night", (800, 533)),
]:
    directory = Path("cadik/tonemapped_images") / image_name
    hdr_brightness = None
    tonemapper_losses = {}

    print(f"Calculating brightness response for {image_name} HDR image")
    for file in directory.iterdir():
        if file.suffix == ".hdr":
            image = dino.read_image(str(file))
            image = dino.resize_image(image, resize_width=width, resize_height=height)
            L = dino.rgb_to_relative_luminance(image)
            print("RMS luminance:", np.mean(L ** 2) ** (1/2))
            hdr_brightness = dino.dn_brightness_model(L)
            break
    print(f"Calculating brightness response for {image_name} tonemapped images")
    for file in tqdm([f for f in directory.iterdir() if f.suffix == ".png"], desc=image_name):
        image = dino.read_image(str(file))
        image = dino.resize_image(image, resize_width=width, resize_height=height)
        L = dino.rgb_to_relative_luminance(image) * 150 # screen white in nits
        tonemapped_brightness = dino.dn_brightness_model(L)
        tonemapper_losses[file.stem.lower()] = MSE(tonemapped_brightness, hdr_brightness)
    tonemapper_brightness_losses[image_name] = tonemapper_losses

print(json.dumps(tonemapper_brightness_losses, indent=2))

Calculating brightness response for indoor HDR image
RMS luminance: 75.247734
Calculating brightness response for indoor tonemapped images


indoor:   0%|          | 0/14 [00:00<?, ?it/s]

Calculating brightness response for outdoor HDR image
RMS luminance: 18.263773
Calculating brightness response for outdoor tonemapped images


outdoor:   0%|          | 0/14 [00:00<?, ?it/s]

Calculating brightness response for night HDR image
RMS luminance: 65.17185
Calculating brightness response for night tonemapped images


night:   0%|          | 0/14 [00:00<?, ?it/s]

{
  "indoor": {
    "ashikhmin02": 0.08278354386648121,
    "chiu93": 0.12536040588933467,
    "choudhury03": 0.0802530651214866,
    "clip": 0.028340586028682824,
    "drago03": 0.04997590105865882,
    "durand02": 0.012024156629625932,
    "fattal02": 0.037911126245719906,
    "lcis99": 0.06808526910746239,
    "pattanaik02": 0.08299925236167595,
    "reinhard02": 0.030539185355232534,
    "schlick94": 0.013501980013097133,
    "tumblin99": 0.025741675428982213,
    "ward94": 0.015433499917657805,
    "ward97": 0.029711270887208766
  },
  "outdoor": {
    "ashikhmin02": 0.05590007909498814,
    "chiu93": 0.11105813449815584,
    "choudhury03": 0.05245374373789234,
    "clip": 0.013468868564267147,
    "drago03": 0.027939809888616298,
    "durand02": 0.004011966111834832,
    "fattal02": 0.08094302874124129,
    "lcis99": 0.043561347612476535,
    "pattanaik02": 0.09162954024491057,
    "reinhard02": 0.012557410192881724,
    "schlick94": 0.010363291966629823,
    "tumblin99": 0.01038

In [2]:
import pandas as pd

ranking_df = pd.read_csv("cadik/ranking_evaluation.csv", header=[0,1])
rating_df = pd.read_csv("cadik/rating_evaluation.csv", header=[0,1])

print(ranking_df)

         indoor                                                  outdoor  \
             tm brightness contrast details colours quality           tm   
0    ashikmin02        8.3      8.0    10.2     8.3     7.6   ashikmin02   
1        chiu93        1.1      2.7     3.0     1.1     1.8       chiu93   
2   choudhury03        5.2      5.9     7.0     5.4     2.2  choudhury03   
3          clip       10.6      7.6     7.6    11.3     8.9         clip   
4       drago03       10.9      9.5     6.9     9.0     8.9      drago03   
5      durand02        8.4      4.7     6.9     4.6     3.5     durand02   
6      fattal02        3.2      5.4     7.4     5.0     5.8     fattal02   
7        lcis99        4.1      6.2     5.4     3.4     4.6       lcis99   
8   pattanaik02       11.1      8.9    12.4     8.6     6.8  pattanaik02   
9    reinhard02       10.8     11.6    10.4    12.5    12.2   reinhard02   
10    schlick94        3.8      7.1     6.2     5.6     9.3    schlick94   
11    tumbli

In [3]:
tm_ranking_dfs = {}

for image_name in ["indoor", "outdoor", "night"]:
    print(image_name)

    headers = ["dino"]
    # print("tone mappers ranked by brightness model:")
    dino = []
    # Ascending order, lower loss is better
    for i, (tm, _) in enumerate(sorted(tonemapper_brightness_losses[image_name].items(), key=lambda x: x[1])):
        # print(f"{i+1}. {tm}")
        dino.append(tm)

    cols = [dino]
    for df, name in [(ranking_df, "ranking"), (rating_df, "rating")]:
        for header in df[image_name].columns[1:]:
            # print(f"tone mappers ranked by Cadik {header} {name}:")
            ranking = []
            # Descending order, higher ranking is better
            for i, tm in enumerate(df.sort_values((image_name, header), ascending=False)[image_name]["tm"].to_list()):
                # print(f"{i+1}. {tm}")
                ranking.append(tm)
            cols.append(ranking)
            headers.append(f"{header}_{name}")

    tm_ranking_df = pd.DataFrame({h: col for h, col in zip(headers, cols, strict=True)})
    tm_ranking_df.to_csv(f"cadik/{image_name}_tm_ranking.csv")
    tm_ranking_dfs[image_name] = tm_ranking_df

indoor
outdoor
night


In [4]:
print(tm_ranking_dfs["indoor"])

           dino brightness_ranking contrast_ranking details_ranking  \
0      durand02          tumblin99       reinhard02     pattanaik02   
1     schlick94        pattanaik02           ward97      reinhard02   
2        ward94            drago03          drago03      ashikmin02   
3     tumblin99         reinhard02        tumblin99          ward97   
4          clip               clip      pattanaik02            clip   
5        ward97             ward97           ward94       tumblin99   
6    reinhard02           durand02       ashikmin02        fattal02   
7      fattal02         ashikmin02             clip     choudhury03   
8       drago03             ward94        schlick94         drago03   
9        lcis99        choudhury03           lcis99        durand02   
10  choudhury03             lcis99      choudhury03       schlick94   
11  ashikhmin02          schlick94         fattal02          lcis99   
12  pattanaik02           fattal02         durand02          ward94   
13    

In [5]:
from scipy.stats import kruskal


def key_sorted_values(dct, normalize=True):
    values = [dct[k] for k in sorted(dct.keys())]
    if normalize:
        max_value = max(*values)
        values = [v/max_value for v in values]
    return values

def key_positions(dct, reverse=False):
    positions = {k: i for i, (k, _) in enumerate(sorted(dct.items(), key=lambda x: x[1], reverse=reverse))}
    return [positions[k] for k in sorted(dct.keys())]

kruskal_results_dfs = {}

for image_name in ["indoor", "outdoor", "night"]:
    print(image_name)
    dino_ratings = tonemapper_brightness_losses[image_name]
    dino_ratings_list = key_positions(dino_ratings, reverse=False)
    df_data = {}
    for header in ranking_df[image_name].columns[1:]:
        rankings = ranking_df[image_name]
        ratings = rating_df[image_name]
        tm_rankings = {rankings.tm[i]: rankings[header][i] for i in range(len(ranking_df.index))}
        tm_ratings = {ratings.tm[i]: ratings[header][i] for i in range(len(rating_df.index))}
        assert len(tm_rankings) == len(tm_ratings)
        assert len(tm_rankings) == len(dino_ratings)
        tm_rankings_list = key_positions(tm_rankings, reverse=True)
        tm_ratings_list = key_positions(tm_ratings, reverse=True)
        statistic, p_value = kruskal(*list(zip(dino_ratings_list, tm_rankings_list, tm_ratings_list)))
        kruskal_results = {"H (DINOS vs rating vs ranking)": statistic, "p (DINOS vs rating vs ranking)": p_value}
        statistic, p_value = kruskal(*list(zip(dino_ratings_list, tm_ratings_list)))
        kruskal_results.update({"H (DINOS vs rating)": statistic, "p (DINOS vs rating)": p_value})
        statistic, p_value = kruskal(*list(zip(dino_ratings_list, tm_rankings_list)))
        kruskal_results.update({"H (DINOS vs ranking)": statistic, "p (DINOS vs ranking)": p_value})
        statistic, p_value = kruskal(*list(zip(tm_rankings_list, tm_ratings_list)))
        kruskal_results.update({"H (rating vs ranking)": statistic, "p (rating vs ranking)": p_value})
        df_data[header] = kruskal_results
    kruskal_results_dfs[image_name] = pd.DataFrame(df_data)

indoor
outdoor
night


In [6]:
for k, v in kruskal_results_dfs.items():
    print(k)
    print(v)
    print("")

indoor
                                brightness   contrast    details    colours  \
H (DINOS vs rating vs ranking)   17.090842  21.255922  13.686691  25.861538   
p (DINOS vs rating vs ranking)    0.195184   0.068019   0.396273   0.017743   
H (DINOS vs rating)              11.452747  15.428571   9.197802  17.802198   
p (DINOS vs rating)               0.572945   0.281367   0.757846   0.165175   
H (DINOS vs ranking)             14.894505  15.013187  11.156044  17.980220   
p (DINOS vs ranking)              0.313985   0.306530   0.597747   0.158272   
H (rating vs ranking)            19.226374  21.303297  20.175824  22.786813   
p (rating vs ranking)             0.116250   0.067142   0.090912   0.044307   

                                  quality  
H (DINOS vs rating vs ranking)  23.058120  
p (DINOS vs rating vs ranking)   0.040984  
H (DINOS vs rating)             14.419780  
p (DINOS vs rating)              0.344959  
H (DINOS vs ranking)            20.175824  
p (DINOS vs ranki